# Oil Price Prediction with Recurrent Neural Networks
Try to predict the prices of brent, based on the daily historical price with the help of the *Recurrent Neural Networks*.

In [18]:
import numpy as np
import pandas as pd

In [19]:
df = pd.read_csv('./datasets/Oil_Prices/brent-daily_csv.csv')

In [20]:
df.head()

,Date,Price
0,1987-05-20,18.63
1,1987-05-21,18.45
2,1987-05-22,18.55
3,1987-05-25,18.60
4,1987-05-26,18.63


In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8116 entries, 0 to 8115
Data columns (total 2 columns):
Date     8116 non-null object
Price    8116 non-null float64
dtypes: float64(1), object(1)
memory usage: 126.9+ KB


**Plan of Attack**
1. Explore the data and derive features from it.
2. Make a model.

## 1. Exploring the data and derive features from it.

In [22]:
from datetime import datetime

In [23]:
# 1. Fromat the date and create features from it
df['Date'] = df['Date'].apply(lambda x: datetime.strptime(x, '%Y-%M-%d'))
df['Year'] = df['Date'].apply(lambda x: x.year)
df['Month'] = df['Date'].apply(lambda x: x.month)
df['Day'] = df['Date'].apply(lambda x: x.day)

In [24]:
df.head()

,Date,Price,Year,Month,Day
0,1987-01-20 00:05:00,18.63,1987,1,20
1,1987-01-21 00:05:00,18.45,1987,1,21
2,1987-01-22 00:05:00,18.55,1987,1,22
3,1987-01-25 00:05:00,18.60,1987,1,25
4,1987-01-26 00:05:00,18.63,1987,1,26
